In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import h5py

In [ ]:
gpu_devices = tf.config.list_physical_devices('GPU')
for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

In [ ]:
with h5py.File("/content/drive/MyDrive/database/val2_dataset.h5", 'r') as h5:
    X = h5['data_full'][:300][:]
    y = h5['label_full'][:300][:]

In [ ]:
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X, y, test_size=0.1, random_state=78)
utils = tf.keras.utils
y = utils.to_categorical(y)

In [ ]:
#model = keras.models.load_model('/content/drive/MyDrive/Trained Model/resNet50.hdfs')
#adam = keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#rms = keras.optimizers.RMSprop()
#model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

In [ ]:
def create_model():
    np.random.seed(1234)
    num_classes = 2
    #base_model = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(160, 160,3))
    base_model = keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(160, 160,3))
    #for layer in base_model.layers:
    #    layer.trainable = False
    cnn = keras.models.Sequential()
    cnn.add(base_model)
    cnn.add(keras.layers.Flatten())
    model = keras.models.Sequential()
    timedistriLay = keras.layers.TimeDistributed(cnn,  input_shape=(30, 160, 160, 3))
    model.add(timedistriLay)
    model.add(keras.layers.LSTM(30 , return_sequences= True))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(90)))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.GlobalAveragePooling1D())
    #model.add(keras.layers.Dense(1024, activation='gelu'))
    #model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(512, activation='gelu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(num_classes, activation="sigmoid"))
    adam = keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    #model.load_weights('/content/drive/MyDrive/Trained Model/VGG19_3.hdf5')
    rms = keras.optimizers.RMSprop()
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])
    return model

In [ ]:
y.shape

In [ ]:
model = create_model()
model.summary()

In [ ]:
results = model.evaluate(X,y,batch_size = 12)
print("test loss, test acc:", results)

In [ ]:
X2 = np.zeros((1, 30, 160, 160, 3), dtype= 'float16')
X2[0][:] = X[0][:]
print(model.predict(X2))

In [ ]:
y[0]

In [ ]:
yy = 2*[1] + 2*[0]
yy = utils.to_categorical(yy)
print(yy)

In [ ]:
import os
from skimage.transform import resize
frames = np.zeros((30, 160, 160, 3), dtype = 'float16')
i = 0
cap = cv2.VideoCapture('/content/drive/MyDrive/database/N_50.mp4')
if cap.isOpened():
    grabbed, frame = cap.read()
else:
    grabbed = False
    print("fail")
frm = resize(frame,(160, 160, 3))
frm = np.expand_dims(frm,axis=0)
if(np.max(frm)>1):
    frm = frm/255.0
frames[i][:] = frm
i += 1
while i < 30:
    grabbed, frame = cap.read()
    frm = resize(frame,(160, 160, 3))
    frm = np.expand_dims(frm,axis=0)
    if(np.max(frm)>1):
        frm = frm/255.0
    frames[i][:] = frm
    i += 1
X2 = np.zeros((1, 30, 160, 160, 3), dtype= 'float16')
X2[0][:] = frames

In [ ]:
print(model.predict(X2))